## 1. Импортировние библиотек.

In [1]:
%pip install pandas plotly scikit-learn nltk torchtext torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# to work with tables
import pandas as pd

# visualization
import plotly.express as px

# to prepare data for the training
from sklearn.utils import shuffle
import re
import time
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# for Neural Network
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# extra settings
pd.options.mode.chained_assignment = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# read data
data = pd.read_csv('meatinfo.csv', delimiter=';')
# for subsequent separation
data = shuffle(data)
display(data)

,text,mtype
9090,жир сырец говяжий. На постоянной основе продаю...,Говядина
14321,"обрезь свиную постность 40-50%, в наличии. дат...",Свинина
16165,Сильвер сайд Парагвай 363руб Сильвер сайд ...,Говядина
4361,"Говядина, тримминг, 80/20 цена 220 руб.",Говядина
11285,"Котлетное мясо свин 60/40 зам котлетное мясо, ...",Свинина
...,...,...
6301,"Говядина, путовый сустав, зам. цена 75 руб.",Говядина
12453,Лапы куриные чищенные,Кура
8974,Жир говяжий топленый в/с РБ Жир говяжий топлен...,Говядина
15556,"Печень свиная Печень свиная, не фасованная, 5-...",Свинина


## 2. Препроцессинг.

Первое, что мы должны сделать - это выполнить первое требование в ТЗ: взять только виды продукции, для которых в датасете есть не менее 500 примеров.

In [ ]:
df_ = data.mtype.value_counts().reset_index()
fig = px.pie(df_, values='mtype', names='index', title='Количество экземпляров')
fig.update_traces(textposition='inside')
fig.show()

В итоге мы будем работать лишь с первыми шестью значениями. Теперь необходимо удалить лишние экземпляры:

In [ ]:
# save shape of the table before the reshaping
shape_before = data.shape
# select just necessary values
data = data[(data['mtype'] == 'Говядина') |
               (data['mtype'] == 'Свинина')  |
               (data['mtype'] == 'Кура') |
               (data['mtype'] == 'Индейка')  |
               (data['mtype'] == 'Баранина') |
               (data['mtype'] == 'Цыпленок')  ]

print('Количество строк до удаления лишних строк: ', shape_before)
print('Количество строк после удаления лишних строк: ', data.shape)

df_ = data.mtype.value_counts().reset_index()
fig = px.pie(df_, values='mtype', names='index', title='Количество экземпляров')
fig.show()

Количество строк до удаления лишних строк:  (17893, 2)
Количество строк после удаления лишних строк:  (16438, 2)


В итоге мы потеряли чуть более тысячи экземпляров из датасета. Теперь необходимо заменить типы продукции в объявлениях на цифры, чтобы затем можно было их интерпретировать в тензоры:

In [ ]:
data['mtype'] = data['mtype'].replace('Говядина', 1)
data['mtype'] = data['mtype'].replace('Свинина', 2)
data['mtype'] = data['mtype'].replace('Кура', 3)
data['mtype'] = data['mtype'].replace('Индейка', 4)
data['mtype'] = data['mtype'].replace('Баранина', 5)
data['mtype'] = data['mtype'].replace('Цыпленок', 6)

Чтобы улучшить качество модели, нам необходимо очистить текст от знаков препинания, восклицания и тд. Для этого воспользуемся библиотекой nltk, которая распологает функции для очистки текста, включая русский язык:

In [ ]:
# loading the dictionary with russian words
stopwords_rus = set(stopwords.words("russian"))

def clean_text(text):
    text = text.lower() # convert to lowercase
    text = re.sub("[^а-я]", " ", text) # leave just letters and gap
    words = [word for word in text.split() if word not in stopwords_rus] # find any stopwords
    text = " ".join(words) # create a new stringa
    return text

str_before_clear = data['text'][0]
# apply the function to all rows in the table
clean_text_df = data['text'].map(clean_text)
data['text'] = clean_text_df

# create dictionary
train_iter = iter(list(zip(data.mtype.to_list(), data.text.to_list())))

print('Строка до очистки: ', str_before_clear)
print('Строка после очистки: ', clean_text_df[0])

Строка до очистки:  12 частей баранина  12 частей баранина
Строка после очистки:  частей баранина частей баранина


Время для разделения датасета на тренировочную (13150 экземпляров - 80%) и тестовую (3288 экземпляров - 20%) части, но стоить отметить, что для обучения нейросети нам необходима выборка для валидации:

In [ ]:
# create list with labels and texts
df = list(zip(data.mtype.to_list(), data.text.to_list()))

train_dataset_df = df[0:11150] # train
valid_dataset_df = df[11150:13150] # validation
test_dataset_df = df[14000:16438] # test

Для успешного обучения модели, нам также нужно токенизировать объявления, чтобы потом иметь возможность, также, как и лейблы, преобразовать их в тензоры. Токенизация происходит посредством присвоения числа слову в словаре, отвечающее за "важность" в тексте. Для этого будет использовать токенайзер от torchtext с русским словарём:

In [ ]:
# loading the tokenizer
tokenizer = get_tokenizer('spacy', language='ru_core_news_sm')

def yield_tokens(data_iter):
    # select text only
    for _, text in data_iter:
        # tokenize the text
        yield tokenizer(text)

# save new vacabulary
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

print("Пример токенизации предложения: \n Предложение: 'продам', 'говядину', 'быстро' \n Токенизированое предложение: ", vocab(['продам', 'говядину', 'быстро']))

Пример токенизации предложения: 
 Предложение: 'продам', 'говядину', 'быстро' 
 Токенизированое предложение:  [19, 63, 3358]


Чтобы модель смогла получить данные для обучения, и, соответственно, для предсказания, нам необходимо создать загрузчика данных посредством встроенного функционала pytorch.

In [ ]:
# tokeninze the text
text_pipeline = lambda x: vocab(tokenizer(x))
# just for optimization
label_pipeline = lambda x: int(x) - 1

def collate_batch(batch):
    # lists for converting them to tensors
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        # append every label, text and offset to lists
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    # converting to tensors
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

Основу нашей модели будет составлять embeddingBag, а также линейный слой для задачи классификации. В конце же необходима инициализация весов.

## 3. Построение модели.

In [ ]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

Определим некоторые гиперпараметры:

In [ ]:
train_iter = list(zip(data.mtype.to_list(), data.text.to_list()))
# number of meat items
num_class = len(set([label for (label, text) in train_iter]))
# vocabulary size
vocab_size = len(vocab)
# batch size
emsize = 64
# attempt to train on cuda
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

Также необходимы функции тренировки и измерения качества модели. В функции тренировки пропишем информационное окно, чтобы было легче отслеживать прогресс обучения модели.

In [ ]:
def train(dataloader):
    # tell our model that we are training the model
    model.train()
    # accuracy rate
    total_acc, total_count = 0, 0
    # batches after which predictions are logged
    log_interval = 500
    # countdown
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        # update optimizer
        optimizer.zero_grad()
        # predict
        predicted_label = model(text, offsets)
        # loss function
        loss = criterion(predicted_label, label)
        # computes dloss/dx
        loss.backward()
        # clip gradient norm
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update values
        optimizer.step()
        # find the true value
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        # add empty tensor (only for calculating accuracy)
        total_count += label.size(0)
        # infolog
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    # tell our model that we are predicting the values
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            # predict the value
            predicted_label = model(text, offsets)
            # loss function
            loss = criterion(predicted_label, label)
            # select true value
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            # add empty tensor (only for calculating accuracy)
            total_count += label.size(0)
    # return accuracy
    return total_acc/total_count

## 4. Обучение модели.

Вот и само обучение: сперва выберем функцию потерь и оптимизатор. Также необходим планировщик, чтобы предотвратить переобучение от больших весов во время обучения.

In [ ]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

# loss function
criterion = torch.nn.CrossEntropyLoss()
# optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
# accuracy
total_accu = None
# train dataset
train_dataloader = DataLoader(train_dataset_df, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# valid dataset
valid_dataloader = DataLoader(valid_dataset_df, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# test dataset
test_dataloader = DataLoader(test_dataset_df, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    # countdown
    epoch_start_time = time.time()
    # train the model
    train(train_dataloader)
    # measure accuracy
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.79s | valid accuracy    0.928 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.79s | valid accuracy    0.939 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.76s | valid accuracy    0.942 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.76s | valid accuracy    0.942 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.82s | valid accuracy    0.949 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.79s |

## 5. Оценка качества модели.

Теперь необходимо измерить качество нашей модели:

In [ ]:
accu_test = evaluate(test_dataloader)
print('Проверка качества модели:')
print('Точность: {:8.2f}%'.format(accu_test))

Проверка качества модели:
Точность:     0.95%


Проверка показывает хорошие результаты, поэтому можно приступить к предсказыванию примеров из ТЗ:

In [ ]:
meat_label = {   1: "Говядина",
                 2: "Свинина",
                 3: "Кура",
                 4: "Индейка", 
                 5: "Баранина",
                 6: "Цыпленок"
             }

def predict(text, text_pipeline):
    with torch.no_grad():
        # clean the text
        text = clean_text(text)
        # create the tensor
        text = torch.tensor(text_pipeline(text))
        # predict
        output = model(text, torch.tensor([0]))
        print(f"Вид продукции в тексте: {meat_label[output.argmax(1).item() + 1]}")

Если вам самим захочется допроверить модель, то можете лишь записать в ковычки предложение для проверки:

In [ ]:
# example = "Напишите предложение здесь"

# predict(example, text_pipeline)

Вид продукции в тексте: Говядина


### Спасибо за внимание!